## Run QSIprep on Multimodhal data


This notebook takes part of the "Multimodhal" project. The final goal is to use the Circular Inference model to modelize the neural pathway of auditory hallucinations in Schizophrenia's patients.

The overall aim of the code here is to provide a robust and replicable pipeline for pre-processing and reconstruction of Diffusion MRI data. X subjects have been integrated in the process. 

The code for each part, pre-processing and reconstruction, is distinct (e.g; each code part will be in its correspondnig titled part) to make it as clear as possible. For both we will use the QSIprep library (more infos here https://qsiprep.readthedocs.io/en/latest/index.html)

- Principal Investigator : Renaud Jardri
- Authors : Alexis Melot, Benoit Noémie
- Laboratory : Lille Neuroscience & Cognition
- Date : 26/01/2026


In [1]:
import os
import sys
import subprocess
sys.path.insert(1, '../tools/')
from utils import extract_b0, add_json_sidecar
import datetime

PRJT_DIR = os.path.abspath(os.path.join(os.getcwd(), "../../../"))
SUBJECTS = [1,2,3]
SESSION = "pre"
DATA_DIR = os.path.join(PRJT_DIR, "data/bids")
FS_LICENSE_PATH = os.path.join(PRJT_DIR, "license.txt")
SYNB0_DIR = os.path.join(PRJT_DIR, "synb0")
QSIPREP_OUTDIR = os.path.join(PRJT_DIR, "results/qsiprep_outputs")
RUN_ID = datetime.datetime.now().strftime("%Y%m%dT%H%M%S")
ACQPARAMS_TXT = "0 -1 0 0.04324205 \n 0 -1 0 0.000"

### SynB0-DISCO requires three specific files per subject to work properly:

- b0.nii.gz: The b=0 image extracted from the DWI (using nibabel).
- T1.nii.gz: The anatomical T1-weighted image.
- acqparams.txt: A text file describing the acquisition sequence.


In [2]:
# Get path of MRI data from SUBJECTS
dwi_paths = []
processes = []
for sub in SUBJECTS:
    dwi_file = os.path.join(DATA_DIR, f"sub-{sub:02d}", f"ses-{SESSION}", "dwi", f"sub-{sub:02d}_ses-{SESSION}_dwi.nii.gz")
    dwi_paths.append(dwi_file)

    # Create output directory for synb0 with datetime if it doesn't exist
    output_sub_dir = os.path.join(SYNB0_DIR, f"{RUN_ID}_sub-{sub:02d}")
    os.makedirs(output_sub_dir, exist_ok=True)
    os.makedirs(os.path.join(output_sub_dir, "outputs"), exist_ok=True)

    # Copie T1w image to output directory
    t1w_output_path = os.path.join(output_sub_dir, f"T1.nii.gz")
    if not os.path.exists(t1w_output_path):
        t1w_path = os.path.join(DATA_DIR, f"sub-{sub:02d}", f"ses-{SESSION}", "anat", f"sub-{sub:02d}_ses-{SESSION}_T1w.nii.gz")
        subprocess.run(["cp", t1w_path, t1w_output_path])

    # Extract b0 image
    b0_output_path = os.path.join(output_sub_dir, "b0.nii.gz")
    if not os.path.exists(b0_output_path):
        extract_b0(dwi_file, b0_output_path)

    # Create acqparams.txt file
    acqparams_path = os.path.join(output_sub_dir, "acqparams.txt")
    if not os.path.exists(acqparams_path):
        with open(acqparams_path, 'w') as f:
            f.write(ACQPARAMS_TXT)

    # Run synb0 with subprocess
    synb0_command = ["docker", "run", "--rm",
                      "-v", f"{output_sub_dir}:/INPUTS/",
                      "-v", f"{output_sub_dir}/outputs:/OUTPUTS/",
                      "-v", f"{FS_LICENSE_PATH}:/extra/freesurfer/license.txt",
                      "--user", f"{os.getuid()}:{os.getgid()}",
                      "leonyichencai/synb0-disco:v3.1",
                      "--no-topup"]
    
    # call cmd line in parallel
    log_path = os.path.join(output_sub_dir, "outputs", "synb0_log.txt")
    with open(log_path, 'w') as log_file:
        process = subprocess.Popen(synb0_command, stdout=log_file, stderr=log_file)
        processes.append((process, sub, output_sub_dir))


Extracted b0 image saved at /home/calculateur2/Documents/AlexisBenoit/dwi_preproc_recon/synb0/20260203T111126_sub-01/b0.nii.gz
Extracted b0 image saved at /home/calculateur2/Documents/AlexisBenoit/dwi_preproc_recon/synb0/20260203T111126_sub-02/b0.nii.gz
Extracted b0 image saved at /home/calculateur2/Documents/AlexisBenoit/dwi_preproc_recon/synb0/20260203T111126_sub-03/b0.nii.gz


In [ ]:
for process, sub, output_sub_dir in processes:
    process.wait()
    
    # create fmap sub-directory
    fmap_dir = os.path.join(DATA_DIR, f"sub-{sub:02d}", f"ses-{SESSION}", "fmap")
    os.makedirs(fmap_dir, exist_ok=True)

    # move synb0 output to fmap directory
    synb0_fmap_output = os.path.join(output_sub_dir, "outputs", "b0_u.nii.gz")
    fmap_output_path = os.path.join(fmap_dir, f"sub-{sub:02d}_ses-{SESSION}_dir-PA_epi.nii.gz")
    subprocess.run(["cp", synb0_fmap_output, fmap_output_path])
    print(f"------ Synb0 output for subject {sub:02d} moved to {fmap_output_path} ------")
    
    # create and add json sidecar for the new fmap
    add_json_sidecar(os.path.join(DATA_DIR, f"sub-{sub:02d}", f"ses-{SESSION}", "dwi", f"sub-{sub:02d}_ses-{SESSION}_dwi.json"),
                     os.path.join(fmap_dir, f"sub-{sub:02d}_ses-{SESSION}_dir-PA_epi.json"))

------ Updated JSON sidecars ------ 
------ Updated JSON sidecars ------ 
------ Updated JSON sidecars ------ 


### Run QSIPrep

In [4]:
# Set allocated resources
NTHREADS = 32
MEM_MB = 64000

In [ ]:
# Run qsiprep with subprocess
qsiprep_command = [
    "docker", "run", "--rm",
    "-v", f"{DATA_DIR}:/mnt/rawdata",
    "-v", f"{QSIPREP_OUTDIR}:/mnt/work",
    "-v", f"{FS_LICENSE_PATH}:/mnt/license.txt",
    "pennlinc/qsiprep:1.1.1",
    "--output-resolution", "2",
    "/mnt/rawdata", "/mnt/work/output",
    "participant",
    "--participant-label"] + [f"{sub:02d}" for sub in SUBJECTS] + [
    "--fs-license-file", "/mnt/license.txt",
    "--omp-nthreads", str(NTHREADS),
    "--mem", str(MEM_MB),
    "--skip-bids-validation",
    "--unringing-method", "mrdegibbs",
    "--work-dir", "/mnt/work/workdir",
    "-vv",
    "--session-id", f"ses-{SESSION}",
    "--stop-on-first-crash"
]
subprocess.run(qsiprep_command)


260203-11:12:22,457 cli INFO:
	 Telemetry system to collect crashes and errors is enabled - thanks for your feedback! Use option ``--notrack`` to opt out.
260203-11:12:29,86 nipype.workflow INFO:
	 
Group	Subject	Sessions
0	02	pre
1	01	pre
2	03	pre
